In [1]:
import os
import requests
import urllib
import json
from datetime import date, datetime
import time
import pandas as pd
from collections import defaultdict
import sqlalchemy
from build_db import connect, build_matches, append_matches,\
    append_player_matches, build_player_matches, build_heroes, build_hero_ranking,\
    append_hero_ranking, build_mmr_estimates, append_mmr_estimates

#  Match Data

Lets go grab the match id of a recent match off dotabuff:
3440629665

In [2]:
match_url = 'https://api.opendota.com/api/matches/{match_id}'
r = requests.get(match_url.format(match_id=3440629665))
content = json.loads(r.content)

# Writing our own queries
OpenDotA implements OpenAPI standard, which allows us to submit our own SQL queries to their postgreSQL database.
Let's start by seeing what tables we can query.

In [2]:
def explorer_request(request):
    '''
    Parameters: request(string)
    Returns: rows(list of outputs)
    '''
    explorer_url = 'https://api.opendota.com/api/explorer?sql='
    request_url = urllib.quote(request)
    r = requests.get(explorer_url + request_url)
    rows = json.loads(r.content)['rows']
    return rows


** Get Schema **

In [85]:
tables_query = '''
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema='public'
    AND table_type='BASE TABLE';
'''


tables = [row['table_name'] for row in explorer_request(tables_query)]
tables

[u'matches',
 u'team_rating',
 u'hero_ranking',
 u'mmr_estimates',
 u'player_matches',
 u'heroes',
 u'public_matches',
 u'queue',
 u'solo_competitive_rank',
 u'competitive_rank',
 u'public_player_matches',
 u'leagues',
 u'items',
 u'team_match',
 u'match_patch',
 u'picks_bans',
 u'teams',
 u'notable_players',
 u'match_logs']

How many matches can a single query return?

In [17]:
public_player_match_query = urllib.quote('''
SELECT * FROM public_player_matches LIMIT 1000
''')
r = requests.get(explorer_url + public_player_match_query)
public_matches_100 = json.loads(r.content)

len(public_matches_100['rows'])

1000

A lot apparently. With a rate limit of 3/s this should be more than adequate. Let's see if we can write a query to return CM matches only. CM is game mode 4 according to dota 2 wiki, though we can confirm that later.

### Querying individual match data
Fields of interest:
- duration: int, length of the match in seconds
- game_mode: int 0 for none, 1 for AP, 2 for CM
- match_id
- picks_bans: list of pick/ban dictionaries { 'is_pick': boolean, 'hero_id': int, 'order': int, 'team': int }
- radiant_win: boolean
- start_time: int, convert to date using date.fromtimestamp(ord)
- match_seq_num: number indicating order of matches


In [18]:
matches_query = urllib.quote('''
SELECT * FROM matches LIMIT 1
''')
r = requests.get(explorer_url + matches_query)
matches = json.loads(r.content)

matches['rows']

[{u'barracks_status_dire': 63,
  u'barracks_status_radiant': 0,
  u'chat': [{u'key': u'gg',
    u'time': 4016,
    u'type': u'chat',
    u'unit': u'iG.Faith'},
   {u'key': u'gg',
    u'time': 4019,
    u'type': u'chat',
    u'unit': u'\u8001\u7537\u5b69'},
   {u'key': u'gg',
    u'time': 4020,
    u'type': u'chat',
    u'unit': u'\u53ef\u8fbe\u9e2d'}],
  u'cluster': 225,
  u'cosmetics': {},
  u'dire_captain': 98887913,
  u'dire_score': 0,
  u'dire_team_complete': 1,
  u'dire_team_id': 1375614,
  u'dire_team_name': None,
  u'duration': 3224,
  u'engine': 0,
  u'first_blood_time': 65,
  u'game_mode': 2,
  u'human_players': 10,
  u'leagueid': 1284,
  u'lobby_type': 1,
  u'match_id': 728669374,
  u'match_seq_num': 656191098,
  u'negative_votes': 10,
  u'objectives': [{u'key': 0,
    u'slot': 8,
    u'time': 867,
    u'type': u'CHAT_MESSAGE_FIRSTBLOOD'},
   {u'slot': 6,
    u'team': 100,
    u'time': 1291,
    u'type': u'CHAT_MESSAGE_TOWER_DENY'},
   {u'slot': 9,
    u'team': 3,
    u'time'

### Querying CM matches
In order to build our query properly, we want to grab all the public matches between the dates
2017-05-15
and 
2017-09-18 where game_mode = 2.

In [6]:
def parse_date(date):
    return int(time.mktime(datetime.strptime(date, '%Y-%m-%d').timetuple()))
start = parse_date('2016-12-12')
end = parse_date('2017-09-18')

In [83]:
cm_query = '''
SELECT match_id, duration,
picks_bans, radiant_win,
match_seq_num
FROM matches 
WHERE start_time BETWEEN {start} AND {end}
ORDER BY match_seq_num ASC
LIMIT 20;
'''.format(start=start, end=end)
explorer_request(cm_query)

[{u'duration': 1969,
  u'match_id': 3180507665,
  u'match_seq_num': 2778110720,
  u'picks_bans': [{u'hero_id': 114,
    u'is_pick': False,
    u'order': 0,
    u'team': 1},
   {u'hero_id': 97, u'is_pick': False, u'order': 1, u'team': 0},
   {u'hero_id': 88, u'is_pick': False, u'order': 2, u'team': 1},
   {u'hero_id': 104, u'is_pick': False, u'order': 3, u'team': 0},
   {u'hero_id': 32, u'is_pick': True, u'order': 4, u'team': 1},
   {u'hero_id': 14, u'is_pick': True, u'order': 5, u'team': 0},
   {u'hero_id': 5, u'is_pick': True, u'order': 6, u'team': 0},
   {u'hero_id': 106, u'is_pick': True, u'order': 7, u'team': 1},
   {u'hero_id': 54, u'is_pick': False, u'order': 8, u'team': 0},
   {u'hero_id': 76, u'is_pick': False, u'order': 9, u'team': 1},
   {u'hero_id': 8, u'is_pick': False, u'order': 10, u'team': 0},
   {u'hero_id': 55, u'is_pick': False, u'order': 11, u'team': 1},
   {u'hero_id': 74, u'is_pick': True, u'order': 12, u'team': 0},
   {u'hero_id': 37, u'is_pick': True, u'order': 1

### Player Data
Later on down the line we may want to add player data to our analysis.

###### player_matches
Useful fields in player_matches: account_id (to join with mmr_estimates), match_id (to join with matches), hero_id

In [89]:
player_request = '''
SELECT * FROM player_matches LIMIT 1;
'''
explorer_request(player_request)

[{u'ability_upgrades_arr': [5023,
   5024,
   5023,
   5025,
   5023,
   5026,
   5023,
   5025,
   5025,
   5025,
   5026],
  u'ability_uses': {u'earthshaker_echo_slam': 2,
   u'earthshaker_enchant_totem': 32,
   u'earthshaker_fissure': 39},
  u'account_id': 112250781,
  u'actions': {u'1': 5965,
   u'10': 348,
   u'11': 11,
   u'15': 4,
   u'16': 21,
   u'19': 16,
   u'2': 118,
   u'3': 5,
   u'33': 348,
   u'4': 474,
   u'5': 74,
   u'6': 4,
   u'7': 4,
   u'8': 86},
  u'additional_units': None,
  u'assists': 7,
  u'backpack_0': 0,
  u'backpack_1': 0,
  u'backpack_2': 0,
  u'buyback_log': [],
  u'camps_stacked': None,
  u'creeps_stacked': None,
  u'damage': {u'illusion_npc_dota_hero_templar_assassin': 634,
   u'npc_dota_badguys_siege': 482,
   u'npc_dota_creep_badguys_melee': 13039,
   u'npc_dota_creep_badguys_melee_upgraded': 1068,
   u'npc_dota_creep_badguys_ranged': 6019,
   u'npc_dota_dark_troll_warlord_skeleton_warrior': 895,
   u'npc_dota_hero_batrider': 955,
   u'npc_dota_hero

###### mmr_estimates
Not sure why the database lists MMR as an estimate.

In [88]:
mmr_request = '''
SELECT * FROM mmr_estimates LIMIT 1;
'''
explorer_request(mmr_request)

[{u'account_id': 327857781, u'estimate': 2324}]

###### hero_ranking
Unclear whether higher or lower is better for hero ranking. Also unclear how the score is calculated, though presumably it's based on a number of factors such as winrate, gpm, kills, deaths, tower damage. Does it only show heroes which have been played a certain number of times?

In [95]:
hero_ranking_request = '''
SELECT * FROM hero_ranking WHERE account_id = 178171791
'''
explorer_request(hero_ranking_request)

[{u'account_id': 178171791, u'hero_id': 2, u'score': 0.00297070883637009},
 {u'account_id': 178171791, u'hero_id': 3, u'score': 0.0333496595784392},
 {u'account_id': 178171791, u'hero_id': 7, u'score': 0.178081511642406},
 {u'account_id': 178171791, u'hero_id': 9, u'score': 0.132611038905244},
 {u'account_id': 178171791, u'hero_id': 14, u'score': 0.0376729461746511},
 {u'account_id': 178171791, u'hero_id': 18, u'score': 0.0333496595784392},
 {u'account_id': 178171791, u'hero_id': 19, u'score': 0.0600666011804639},
 {u'account_id': 178171791, u'hero_id': 21, u'score': 0.0665537737648762},
 {u'account_id': 178171791, u'hero_id': 26, u'score': 0.0333496595784392},
 {u'account_id': 178171791, u'hero_id': 27, u'score': 0.120878180075593},
 {u'account_id': 178171791, u'hero_id': 28, u'score': 0.00345908790462868},
 {u'account_id': 178171791, u'hero_id': 29, u'score': 0.00246191057805359},
 {u'account_id': 178171791, u'hero_id': 30, u'score': 0.0719351228098365},
 {u'account_id': 178171791, u

###### heroes
Hero information may be useful when we get to visualization. We could also allow the user to specify what roles they want the recommender to choose from or what attr, attack type they want.

In [96]:
hero_query = '''
SELECT * FROM heroes LIMIT 1
'''
explorer_request(hero_query)

[{u'attack_type': u'Melee',
  u'id': 44,
  u'legs': 2,
  u'localized_name': u'Phantom Assassin',
  u'name': u'npc_dota_hero_phantom_assassin',
  u'primary_attr': u'agi',
  u'roles': [u'Carry', u'Escape']}]

###### team_match
Not really sure how we could use team match information. It may be an easier way to tell which team a player is on for a given match.

In [97]:
team_match_query = '''
SELECT * FROM team_match LIMIT 1
'''
explorer_request(team_match_query)

[{u'match_id': 672230339, u'radiant': True, u'team_id': 260085}]

If we want to get hero pickrate and hero winrate by player and use those to calculate our own metrics we would have to query 

# Writing to our database

In [100]:
cm_rows = explorer_request(cm_query)
cm_dict = defaultdict(list)
for key in cm_rows[0]:
    for row in cm_rows:
        cm_dict[key].append(row[key])
df = pd.DataFrame(data=cm_dict)

In [101]:
df.head()

,duration,match_id,match_seq_num,picks_bans,radiant_win
0,1969,3180507665,2778110720,"[{u'team': 1, u'hero_id': 114, u'order': 0, u'...",True
1,1356,3180707501,2778275515,"[{u'team': 1, u'hero_id': 57, u'order': 0, u'i...",True
2,1705,3180807310,2778377937,"[{u'team': 1, u'hero_id': 32, u'order': 0, u'i...",False
3,2338,3180828452,2778416570,"[{u'team': 1, u'hero_id': 25, u'order': 0, u'i...",False
4,2570,3180953074,2778549215,"[{u'team': 0, u'hero_id': 99, u'order': 0, u'i...",True


In [9]:
with open(os.path.expanduser('~/.pgpass')) as f:
    for line in f:
        host, port, db, user, password = [x.strip() for x in line.split(':')]
        if db == 'dota-draft-test':
            break
conn, meta = connect(user=user, password=password, db=db, host=host, port=port)

# Unnesting JSON and Joining with Player data

In [150]:
query = '''
SELECT match_id, unnest(picks_bans) FROM matches LIMIT 21
'''
explorer_request(query)

[{u'match_id': 728669374,
  u'unnest': {u'hero_id': 89, u'is_pick': False, u'order': 0, u'team': 1}},
 {u'match_id': 728669374,
  u'unnest': {u'hero_id': 77, u'is_pick': False, u'order': 1, u'team': 0}},
 {u'match_id': 728669374,
  u'unnest': {u'hero_id': 69, u'is_pick': False, u'order': 2, u'team': 1}},
 {u'match_id': 728669374,
  u'unnest': {u'hero_id': 58, u'is_pick': False, u'order': 3, u'team': 0}},
 {u'match_id': 728669374,
  u'unnest': {u'hero_id': 65, u'is_pick': True, u'order': 4, u'team': 1}},
 {u'match_id': 728669374,
  u'unnest': {u'hero_id': 106, u'is_pick': True, u'order': 5, u'team': 0}},
 {u'match_id': 728669374,
  u'unnest': {u'hero_id': 9, u'is_pick': True, u'order': 6, u'team': 0}},
 {u'match_id': 728669374,
  u'unnest': {u'hero_id': 10, u'is_pick': True, u'order': 7, u'team': 1}},
 {u'match_id': 728669374,
  u'unnest': {u'hero_id': 103, u'is_pick': False, u'order': 8, u'team': 1}},
 {u'match_id': 728669374,
  u'unnest': {u'hero_id': 78, u'is_pick': False, u'order': 

# Testing Functions
Check if your DB is being updated

In [23]:
build_matches(conn)
%timeit match_seq_range = append_matches(conn, 20, start, end)

1 loop, best of 3: 232 ms per loop


In [24]:
build_player_matches(conn)
%timeit append_player_matches(conn, start, end, *match_seq_range)
# player_matches table should have 40 rows now

1 loop, best of 3: 277 ms per loop


In [200]:
build_heroes(conn)

In [25]:
build_hero_ranking(conn)
%timeit append_hero_ranking(conn, start, end, *match_seq_range)

1 loop, best of 3: 702 ms per loop


In [26]:
build_mmr_estimates(conn)
%timeit append_mmr_estimates(conn, start, end, *match_seq_range)

1 loop, best of 3: 344 ms per loop


In [11]:
match_seq_range

(2780546657, 2782858339)

In [7]:
url = 'https://api.opendota.com/api/explorer?sql='
query = '''
SELECT COUNT(*) FROM matches WHERE
start_time BETWEEN {start} AND {end}
AND game_mode = 2
'''.format(start=start, end=end)
json.loads(requests.get(url + urllib.quote(query)).content)['rows']

[{u'count': 7105}]

20 matches with all player data takes ~ 3 seconds. Time will probably decrease the more queries it runs, as it populates the player database

In [4]:
query = '''
SELECT picks_bans FROM matches WHERE game_mode = 2 LIMIT 10;
'''
explorer_request(query)

[{u'picks_bans': [{u'hero_id': 89, u'is_pick': False, u'order': 0, u'team': 1},
   {u'hero_id': 77, u'is_pick': False, u'order': 1, u'team': 0},
   {u'hero_id': 69, u'is_pick': False, u'order': 2, u'team': 1},
   {u'hero_id': 58, u'is_pick': False, u'order': 3, u'team': 0},
   {u'hero_id': 65, u'is_pick': True, u'order': 4, u'team': 1},
   {u'hero_id': 106, u'is_pick': True, u'order': 5, u'team': 0},
   {u'hero_id': 9, u'is_pick': True, u'order': 6, u'team': 0},
   {u'hero_id': 10, u'is_pick': True, u'order': 7, u'team': 1},
   {u'hero_id': 103, u'is_pick': False, u'order': 8, u'team': 1},
   {u'hero_id': 78, u'is_pick': False, u'order': 9, u'team': 0},
   {u'hero_id': 74, u'is_pick': False, u'order': 10, u'team': 1},
   {u'hero_id': 29, u'is_pick': False, u'order': 11, u'team': 0},
   {u'hero_id': 53, u'is_pick': True, u'order': 12, u'team': 0},
   {u'hero_id': 87, u'is_pick': True, u'order': 13, u'team': 1},
   {u'hero_id': 68, u'is_pick': True, u'order': 14, u'team': 0},
   {u'hero_

In [17]:
s = map(lambda x: x['picks_bans'], explorer_request(query))

In [18]:
df = pd.DataFrame({'picks_bans': s})

In [19]:
df.head()

,picks_bans
0,"[{u'team': 1, u'hero_id': 89, u'order': 0, u'i..."
1,"[{u'team': 0, u'hero_id': 39, u'order': 0, u'i..."
2,"[{u'team': 0, u'hero_id': 73, u'order': 0, u'i..."
3,"[{u'team': 1, u'hero_id': 11, u'order': 0, u'i..."
4,"[{u'team': 0, u'hero_id': 73, u'order': 0, u'i..."


In [16]:
df.to_sql('picks_bans', conn, if_exists='replace', dtype={'picks_bans' : sqlalchemy.types.JSON})

In [20]:
query = '\n    SELECT match_id, duration,\n    picks_bans, radiant_win,\n    match_seq_num\n    FROM matches\n    WHERE start_time BETWEEN 1351237677 AND 1506284352\n    AND match_seq_num > 0\n    AND game_mode = 2\n    ORDER BY match_seq_num ASC\n    LIMIT 20;\n    '
explorer_request(query)

[{u'duration': 2975,
  u'match_id': 53510908,
  u'match_seq_num': 51269400,
  u'picks_bans': None,
  u'radiant_win': True},
 {u'duration': 2460,
  u'match_id': 53534174,
  u'match_seq_num': 51292409,
  u'picks_bans': None,
  u'radiant_win': True},
 {u'duration': 1907,
  u'match_id': 53609860,
  u'match_seq_num': 51357335,
  u'picks_bans': None,
  u'radiant_win': False},
 {u'duration': 3620,
  u'match_id': 53641581,
  u'match_seq_num': 51399649,
  u'picks_bans': None,
  u'radiant_win': True},
 {u'duration': 1548,
  u'match_id': 53723251,
  u'match_seq_num': 51459655,
  u'picks_bans': None,
  u'radiant_win': False},
 {u'duration': 1840,
  u'match_id': 53733166,
  u'match_seq_num': 51469957,
  u'picks_bans': None,
  u'radiant_win': False},
 {u'duration': 1094,
  u'match_id': 53745978,
  u'match_seq_num': 51475768,
  u'picks_bans': None,
  u'radiant_win': True},
 {u'duration': 1780,
  u'match_id': 53776454,
  u'match_seq_num': 51508352,
  u'picks_bans': None,
  u'radiant_win': True},
 {u'd

In [31]:
query = 'SELECT COUNT(*) FROM matches WHERE game_mode = 2 and picks_bans IS NOT NULL'
explorer_request(query)

[{u'count': 46347}]

In [28]:
query = 'SELECT COUNT(*) FROM matches WHERE game_mode = 2 AND picks_bans IS NULL'
explorer_request(query)

[{u'count': 855}]

In [3]:
query = 'SELECT start_time FROM matches LIMIT 5'
explorer_request(query)

[{u'start_time': 1338225112},
 {u'start_time': 1338229643},
 {u'start_time': 1338234128},
 {u'start_time': 1338314072},
 {u'start_time': 1338405104}]